In [1]:
import yaml
from pathlib import Path
import os

In [2]:
root = Path('~/FCA_Fetal_Skin_priv').expanduser()
os.chdir(root)
!pwd

/home/jovyan/FCA_Fetal_Skin_priv


List of packages extracted from "~/FCA_Fetal_Skin_priv/code/notebooks/figures/fig2D.ipynb" as a starting reference for creating the base conda env (FCA_base) for the project.

In [138]:
def load_env(yml_file):
    """Loads the contents of a conda environment into a dictionary.
    --------------------------------------------------------------------
    Parameters:
    
    - yml_file: Path to the query environment's .yml file. (str)
    ----------
    outputs:
    
    1. env : {package_1 : version, ... , package_n : version}
    2. channels : [list, of, channels]
    """
    
    from collections import defaultdict
    from collections import OrderedDict
    from colorama import Fore, Style
    from math import floor, ceil
    
    assert type(yml_file) == str, "yml_file MUST be of type str!"
    
    env = defaultdict(list)
    channels = []
    
    with Path(yml_file).open(mode='r') as FCA_pakgs:
        package_list = yaml.load(FCA_pakgs, Loader=yaml.FullLoader)
        for item, doc in package_list.items():
            if 'dependencies' in item:
                for packg in doc:
                    try:
                        pack = '='.join(packg.split('==')).split('=')
                        try:
                            env['package'].append(pack[0])
                            env['version'].append( pack[1])
                            env['build'].append(pack[2])
                        except IndexError:
                            env['version'].append('unspecified')
                            env['build'].append('unspecified')
                    except AttributeError:
                        if 'pip' in packg:
                            for pip in packg['pip']:
                                pack = '='.join(pip.split('==')).split('=')
                                if pack[0] == pack[-1]:
                                    env['package'].append(pack[0])
                                    env['version'].append('unspecified')
                                    env['build'].append('unspecified')
                                else:
                                    env['package'].append(f'{pack[0]} **pip**')
                                    env['version'].append(pack[1])
                                    env['build'].append('unspecified')
                        else:
                            print(f'{Fore.RED}{packg} includes unrecognised sources of installation. Skipping the rest of the .yml file!{Style.RESET_ALL}')
                            break
            elif 'channels' in item:
                for ch in doc:
                    channels.append(ch)
                
    return env, channels

In [34]:
class PackageNotFound(Exception):
    """Raised when the target packages are not found"""
    pass

In [ ]:
class PackageNotFound(Exception):
    """Raised when the target packages are not found"""
    def __init__(self, message, payload=None):
        self.message = message
        self.payload = payload # you could add more args
    def __str__(self):
        return str(self.message) # __str__() obviously expects a string to be returned, so make sure not to send any other data types and fetch them like:

try:
    raise CustomExceptionName("Very bad mistake.", "Forgot upgrading from Python 1")
except CustomExceptionName as error:
    print(str(error)) # Very bad mistake
    print("Detail: {}".format(error.payload)) # Detail: Forgot upgrading from Python 1


In [34]:
def crossref_2_envs(path_to_env1, path_to_env2, packages=None, diff_only=False, compare_build=False):
    """Takes .yml file from two envs and cross-references the packages
    and version, if check_version is set to True."""
    
    from collections import OrderedDict
    from colorama import Fore, Style
    from math import floor, ceil
    
    assert type(path_to_env1) == str, "path_to_env1 must be a string"
    assert type(path_to_env2) == str, "path_to_env2 must be a string"
    if type(packages) == list:
        assert all(type(pckg) == str for pckg in packages), "All specified packages in packages must of type string!"
    
    env1, channels1 = load_env(path_to_env1)
    env2, channels2 = load_env(path_to_env2)
    if compare_build:
        env1 = {p:''.join([env1['version'][i], ''.join([' ' for i in range(16-len(env1['version'][i]))]), env1['build'][i]]) for i, p in enumerate(env1['package'])}
        env2 = {p:''.join([env2['version'][i], ''.join([' ' for i in range(16-len(env2['version'][i]))]), env2['build'][i]]) for i, p in enumerate(env2['package'])}
    else:
        env1 = {p:env1['version'][i] for i, p in enumerate(env1['package'])}
        env2 = {p:env2['version'][i] for i, p in enumerate(env2['package'])}

    env1_name = path_to_env1.split('/')[-1].split('.')[0]
    env2_name = path_to_env2.split('/')[-1].split('.')[0]
    crossref_diff = dict()
    for p in env1:
        if p not in env2.keys():
            crossref_diff[p] = 'env1'
        else:
            crossref_diff[p] = 'both'
    for p in env2:
        if p not in env1.keys():
            crossref_diff[p] = 'env2'
        else:
            continue
    crossref_diff = OrderedDict(sorted(crossref_diff.items()))
    if packages:
        if type(packages) == list:
            packages = [pckg.lower() for pckg in packages]
            for pckg in packages:
                if (pckg not in crossref_diff.keys()) and (f'{pckg} **pip**' not in crossref_diff.keys()):
                    raise PackageNotFound(f'{pckg} not found in either of the environments!')
        elif type(packages) == str:
            packages = packages.lower()
            if (packages not in crossref_diff.keys()) and (f'{packages} **pip**' not in crossref_diff.keys()):
                    raise PackageNotFound(f'{packages} not found in either of the environments!')
        else:
            raise TypeError(f'packages must be of type str or list!')

    p_max = 0
    v_max = max([len(env1_name), len(env2_name)])
    for p in crossref_diff:
        if len(p) > p_max:
            p_max = len(p)
        try:
            if len(env1[p]) > v_max:
                v_max = len(env1[p])
        except:
            continue
    p_max_spaces = ''.join([' ' for l in range(p_max+9)])
    v_max_spaces = ''.join([' ' for l in range(((v_max-len(env1_name))+15))])
    equal = True
    for ind, p in enumerate(crossref_diff):
        if crossref_diff[p] != 'both':
            equal = False
        else:
            if env1[p] != env2[p]:
                equal = False
    if equal:
        return print(f"\n{Fore.GREEN} No differences found between the two environments. Congratulations!{Style.RESET_ALL}\n")
    for ind, p in enumerate(crossref_diff):
        if ind == 0:
            print('\n')
            if compare_build:
                null = 'null            null'
                env1_name_start = str(f"{''.join([' ' for s in range(len(p_max_spaces))])}" + 
                                    f"{''.join([' ' for s in range(floor((v_max/2)-(ceil(2*(len(env1_name)/3)))))])}")
                env1_name_end = env1_name_start + ''.join([' ' for s in env1_name])
                env2_start = f"{''.join([' ' for s in range((len(p_max_spaces)+v_max+15))])}"
                env2_name_start = str(''.join([' ' for s in range(len(env2_start) - len(env1_name_end))]) + f"{''.join([' ' for s in range((floor(max([len(v) for v in env2.values()])/2)-len(env2_name)))])}")
                print(f"{env1_name_start}{Fore.CYAN}{env1_name}{Style.RESET_ALL}"
                      f"{env2_name_start}{Fore.CYAN}{env2_name}{Style.RESET_ALL}")
                print('')
                # 10 for 6 spaces used between first version and build (below) plus the length of 'build' (i.e. 5 characters) -1.
                print(f"{p_max_spaces}{Fore.BLUE}version{Style.RESET_ALL}"
                      f"{''.join([' ' for s in range(16-len('build'))])}"
                      f"{Fore.BLUE}build{Style.RESET_ALL}"
                      f"{''.join([' ' for s in range(len(env2_start)-len(p_max_spaces)-max([len(v.split(' ')[0]) for v in env1.values()])-len('verion'+'build'))])}"
                      f"{Fore.BLUE}version{Style.RESET_ALL}{''.join([' ' for s in range(max([len(v.split(' ')[0]) for v in env2.values()]))])}"
                      f"{Fore.BLUE}build{Style.RESET_ALL}")
            else:
                null = 'null'
                print(f'{p_max_spaces}{Fore.CYAN}{env1_name}{Style.RESET_ALL}{v_max_spaces}{Fore.CYAN}{env2_name}{Style.RESET_ALL}')
            print('')
        if crossref_diff[p] == 'both':
            pckg_spaces = ''.join([' ' for l in range((p_max+10)-len(p))])
            version_spaces = ''.join([' ' for l in range((v_max+15)-len(env1[p]))])
            if env1[p] == env2[p]:
                if diff_only:
                    continue
                if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                    if '**pip**' not in p:
                        print(f'{p}{pckg_spaces}{Fore.GREEN}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}"
                              f"{Fore.GREEN}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}")
                else:
                    if '**pip**' not in p:
                        print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.GREEN}{env1[p]}{Style.RESET_ALL}'
                              f'{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                              f"{pckg_spaces}{Fore.GREEN}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.GREEN}{env2[p]}{Style.RESET_ALL}")
            else:
                if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                    if '**pip**' not in p:
                        print(f'{p}{pckg_spaces}{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}"
                              f"{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}")
                else:
                    if '**pip**' not in p:
                        print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}'
                              f'{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}')
                    else:
                        print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                              f"{pckg_spaces}{Fore.YELLOW}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.YELLOW}{env2[p]}{Style.RESET_ALL}")
            header = False
        elif crossref_diff[p] == 'env1':
            pckg_spaces = ''.join([' ' for l in range((p_max+10)-len(p))])
            version_spaces = ''.join([' ' for l in range((v_max+15)-len(env1[p]))])
            if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                if '**pip**' not in p:
                    print(f'{p}{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}')
                else:
                    print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}"
                          f"{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}")
            else:
                if '**pip**' not in p:
                    print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}'
                          f'{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}')
                else:
                    print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                          f"{pckg_spaces}{Fore.RED}{env1[p]}{Style.RESET_ALL}{version_spaces}{Fore.RED}{null}{Style.RESET_ALL}")
            header = False
        elif crossref_diff[p] == 'env2':
            pckg_spaces = ''.join([' ' for l in range((p_max+10)-len(p))])
            version_spaces = ''.join([' ' for l in range((v_max+15)-len(null))])
            if (packages==None) or ((type(packages)==str) and (p!=packages)) or ((type(packages)==list) and (not any(p == pckg for pckg in packages)) and (not any(p.split(' **pip**')[0] == pckg for pckg in packages))):
                if '**pip**' not in p:
                    print(f'{p}{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}')
                else:
                    print(f"{p.split(' **pip**')[0]}{Fore.RED} **pip**{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}"
                          f"{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}")
            else:
                if '**pip**' not in p:
                    print(f'{Fore.MAGENTA}{p}{Style.RESET_ALL}{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}'
                          f'{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}')
                else:
                    print(f"{Fore.MAGENTA}{p.split(' **pip**')[0]}{Style.RESET_ALL}{Fore.RED} **pip**{Style.RESET_ALL}"
                          f"{pckg_spaces}{Fore.RED}{null}{Style.RESET_ALL}{version_spaces}{Fore.RED}{env2[p]}{Style.RESET_ALL}")
            header = False

In [54]:
[str(subprocess.run('bash -c "conda env list"', capture_output=True, shell=True)).split(',')[-2].replace(" ", "").split('\\')]

["stdout=b'#condaenvironments:",
 'n#',
 'nFCA/home/jovyan/my-conda-envs/FCA',
 'nFCA_base/home/jovyan/my-conda-envs/FCA_base',
 'nFCA_test/home/jovyan/my-conda-envs/FCA_test',
 'nVCS/home/jovyan/my-conda-envs/VCS',
 'ncircos/home/jovyan/my-conda-envs/circos',
 'nenv_manage/home/jovyan/my-conda-envs/env_manage',
 'nfetal_skin/home/jovyan/my-conda-envs/fetal_skin',
 'nmetaflow/home/jovyan/my-conda-envs/metaflow',
 'nmilo/home/jovyan/my-conda-envs/milo',
 'nmonocle3/home/jovyan/my-conda-envs/monocle3',
 'nqc/home/jovyan/my-conda-envs/qc',
 'ntestenv/home/jovyan/my-conda-envs/testenv',
 'nbase*/opt/conda',
 'nr-reticulate/opt/conda/envs/r-reticulate',
 'n',
 "n'"]

In [123]:
def build_sub_env(yml_file, packages, new_env_name=None, mamba =True, overide_env=False, prefix=None, constraint='exact', image=False):
    """Takes in a yml file for the query env 
    and creates a new environment with additional
    specified packages.
    ----------------------------------------------------------
    Parameters:
    
    """
    
    import subprocess
    import time
    
    assert type(yml_file) == str, "'yml_file' must be a str"
    assert (type(packages) == list) or (type(packages) == dict), "'packages' " + \
    "must be either a dict of package:versions or a list of package names."
    constraint_type = {'fuzzy' : '=', 'exact' : '==', 'GoE' : '>=', 
                       'greater' : '>', 'SoE' : '<=', 'smaller' : '<'}
    assert constraint in list(constraint_type.keys()), f"'constraint' must be set " + \
    "to one of the {list(constraint_type.keys())}. See the docstring for more info."
    if type(packages) == list:
        assert all(type(p) == str for p in packages), "All specified packages " + \
        "in packages must of type str!"
    elif type(packages) == dict:
        assert all(type(p) == str for p in packages.keys()), "All " + \
        "specified package names in packages must of type str!"
        assert all(type(v) == str for v in packages.values()), "All \
        specified package versions in packages must of type str!"
        assert all(any(v.startswith(c) for c in list(constraint_type.values())) \
                   for v in packages.values() if v!=''), "All specified versions " + \
                   "in packages must of type str!"
    
    
    ext_name = yml_file.split('/')[-1].split('.')[0]
    all_envs = str(subprocess.run('bash -c "conda env list"', 
                                  capture_output=True, 
                                  shell=True))
    if not new_env_name:
        print(f"No explicit name provided for the new environment. Falling back " 
              f"to the .yml file's name: '{ext_name}'.")
        new_env_name = ext_name
    if new_env_name in all_envs:
        assert overide_env==True, f"{new_env_name} env already exists! If you " + \
        "wish to overwrite this env, you must set 'overide_env' to True."
        print(f"Removing the exisitng {new_env_name}, you have 10 seconds to halt"
              "this process!")
        time.sleep(10)
        subprocess.run(f"bash -c 'conda env remove -n {new_env_name} -y'", 
                       shell=True)
    
    env, channels = load_env(yml_file)
    env = {p:env['version'][i] for i, p in enumerate(env['package'])}
    channels = [f'-c {ch}' for ch in channels]
    command = []
    if mamba:
        command.append(f'mamba create -n {new_env_name} -y')
    else:
        command.append(f'conda create -n {new_env_name} -y')
    command.extend(channels)
    
    if image:
        command.extend(['=='.join([s.split(' **pip**')[0], env['version'][i]]) for i, s in enumerate(env['package'])])
    
    if type(packages)==dict:
        for pack in packages:
            for s in constraint_type.values():
                pack = pack.split(s)[-1]
            if packages[pack] == '':
                try:
                    command.append(f'{pack}{constraint_type[constraint]}{env[pack]}')
                except KeyError:
                    try:
                        command.append(f"{pack}{constraint_type[constraint]}{env[f'{pack} **pip**']}")
                    except KeyError:
                        command.append(pack)
            else:
                command.append(f'{pack}{packages[pack]}')
    else:
        for pack in packages:
            for s in constraint_type.values():
                pack = pack.split(s)[-1]
            try:
                command.append(f'{pack}{constraint_type[constraint]}{env[pack]}')
            except KeyError:
                try:
                    command.append(f"{pack}{constraint_type[constraint]}{env[f'{pack} **pip**']}")
                except KeyError:
                    command.append(pack)
            
    command = ' '.join(command)
    subprocess.run(f"bash -c '{command}'", 
                   shell=True)
    

In [ ]:
def rebuild_env(yml_file, packages, mamba=True):
    """"""
    import johnnydep
    
    

In [4]:
import johnnydep, sys, os

In [5]:
import os, sys

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

In [31]:
def get_pip_deps(pack_name):
    """Get requirements of a pip package using johnnydep package."""
    
    from colorama import Fore, Style
    
    assert type(pack_name)==str
    
    with HiddenPrints():
        requirements = johnnydep.JohnnyDist(pack_name).requires
    print(f"{Fore.RED}{pack_name}{Style.RESET_ALL} requires : {requirements}")
    
    return requirements

In [7]:
test_jd_reqs = get_pip_deps('celltypist')

['click>=7.1.2', 'leidenalg>=0.8.3', 'numpy>=1.19.0', 'openpyxl>=3.0.4', 'pandas>=1.0.5', 'requests>=2.23.0', 'scanpy>=1.7.0', 'scikit-learn>=0.24.1']


In [127]:
johnnydep.JohnnyDist('pandas==1.5.0').requires

2022-11-30 09:31.48 [info     ] init johnnydist                dist=pandas==1.5.0 parent=None
2022-11-30 09:31.48 [debug    ] fetching best wheel            dist=pandas==1.5.0
2022-11-30 09:31.48 [debug    ] created scratch                dist_name=pandas==1.5.0 tmpdir=/tmp/tmp7vuh7n25
2022-11-30 09:31.48 [debug    ] wheeling and dealing           args=/home/jovyan/my-conda-envs/env_manage/bin/python -m pip wheel -vvv --no-deps --no-cache-dir --disable-pip-version-check --progress-bar=off pandas==1.5.0 scratch_dir=/tmp/tmp7vuh7n25/tmp78yhj8uz
2022-11-30 09:31.49 [debug    ] wheel command completed ok     dist_name=pandas==1.5.0
2022-11-30 09:31.49 [debug    ] computing checksum             algorithm=md5 target=/tmp/tmp7vuh7n25/tmp78yhj8uz/pandas-1.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
2022-11-30 09:31.49 [debug    ] computed checksum              result=6fb3f1e0663a6f778a75e123e635e3c0
2022-11-30 09:31.49 [debug    ] finding import names
2022-11-30 09:31.49 [de

['numpy>=1.21.0', 'python-dateutil>=2.8.1', 'pytz>=2020.1']

In [117]:
def seperate_pck_name(pack_version_str):
    """Takes a package-version string, e.g. 'scikit-learn>=0.24.1', 
    and returns the package name, e.g. scikit-learn"""
    
    import re
    
    if any(c in pack_version_str for c in ['<', '=', '>', ',', ' ']):
        pack_name = re.search(r'.*?(?=(<|=|>|,))', pack_version_str).group(0)
        print(_str)
        return , _str.group(2)
    else:
        return pack_version_str

In [124]:
 re.search(r'(?<=(<|=|>|,)).*', 'anndata>=0.7.4').group(0)

'=0.7.4'

In [131]:
get_pip_deps(missing_pack)

"'certifi>=2017.4.17'"

In [118]:
compiled_packs = []
for p in test_jd_reqs:
    compiled_packs.extend(get_pip_deps(p))

for p in compiled_packs:
    print(seperate_pck_name(p))

click>=7.1.2 requires : []
leidenalg>=0.8.3 requires : ['igraph<0.11,>=0.10.0']
numpy>=1.19.0 requires : []
openpyxl>=3.0.4 requires : ['et-xmlfile']
pandas>=1.0.5 requires : ['numpy>=1.21.0', 'python-dateutil>=2.8.1', 'pytz>=2020.1']
requests>=2.23.0 requires : ['certifi>=2017.4.17', 'charset-normalizer<3,>=2', 'idna<4,>=2.5', 'urllib3<1.27,>=1.21.1']
scanpy>=1.7.0 requires : ['anndata>=0.7.4', 'h5py>=3', 'joblib', 'matplotlib>=3.4', 'natsort', 'networkx>=2.3', 'numba>=0.41.0', 'numpy>=1.17.0', 'packaging', 'pandas>=1.0', 'patsy', 'scikit-learn>=0.22', 'scipy>=1.4', 'seaborn', 'session-info', 'statsmodels>=0.10.0rc2', 'tqdm', 'umap-learn>=0.3.10']
scikit-learn>=0.24.1 requires : ['joblib>=1.0.0', 'numpy>=1.17.3', 'scipy>=1.3.2', 'threadpoolctl>=2.0.0']
<re.Match object; span=(0, 6), match='igraph'>


IndexError: no such group

In [ ]:
import subprocess

def execute(cmd):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line 
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

prob_tup = tuple()
conflicts = []
missing_packages = []
pip_only = []
pip_only_deps = []
try:
    for line in execute(["mamba", "install", "-y", "-c", "conda-forge", "pandas==1.5.1", "numpy==1.9.3", "python==3.7"]):
        
        else:
            print(path, end="")
except:
    if 'nothing provides requested' in line:
        try:
            missing_pack = re.search(f"(?<=nothing provides requested ).*", line).group(0)
            pip_only_deps.extend(get_pip_deps(missing_pack))
            pip_only.append(missing_pack)
        except CalledProcessError:
            print(f"{missing_pack} is not found nor by anaconda neither by pip.")
            print(f"Make sure you have provided the correct spelling.")
            print("Exiting!")
    elif 'requires' in path:
        problem_search = re.search(f"(?<=package).*(?=, but)", path)
        prob1 = test_search.group(0).split(f'-')[:-1]
        prob2 = test_search.group(0).split(f' requires ')[-1]
        print(f"{'=='.join(prob1) + ' requires ' + prob2}")
        print("Try provodng fuzzy constraint for the requested package (e.g. '=ver.sion.x'), if the specified patch 'x' is not essential.")
        print("Alternatively, set constraint='fuzzy' if the env packages' patches are not essential")
        print('Exiting!')

In [45]:
print(len([seperate_pck_name(p) for p in compiled_packs]))
print(len(set([seperate_pck_name(p) for p in compiled_packs])))

31
27


In [112]:
import johnnydep 

johnnydep.JohnnyDist('requestss').requires

2022-11-29 14:05.49 [info     ] init johnnydist                dist=requestss parent=None
2022-11-29 14:05.49 [debug    ] fetching best wheel            dist=requestss
2022-11-29 14:05.49 [debug    ] created scratch                dist_name=requestss tmpdir=/tmp/tmpc6tgv7c8
2022-11-29 14:05.49 [debug    ] wheeling and dealing           args=/home/jovyan/my-conda-envs/env_manage/bin/python -m pip wheel -vvv --no-deps --no-cache-dir --disable-pip-version-check --progress-bar=off requestss scratch_dir=/tmp/tmpc6tgv7c8/tmp3647w2zd
2022-11-29 14:05.50 [warning  ] Created temporary directory: /tmp/pip-ephem-wheel-cache-uvzuk_pg
Created temporary directory: /tmp/pip-build-tracker-31n_55ea
Initialized build tracking at /tmp/pip-build-tracker-31n_55ea
Created build tracker: /tmp/pip-build-tracker-31n_55ea
Entered build tracker: /tmp/pip-build-tracker-31n_55ea
Created temporary directory: /tmp/pip-wheel-03ylydy1
Created temporary directory: /tmp/pip-ephem-wheel-cache-sei5av_o
1 location(s) to se

CalledProcessError: Command '['/home/jovyan/my-conda-envs/env_manage/bin/python', '-m', 'pip', 'wheel', '-vvv', '--no-deps', '--no-cache-dir', '--disable-pip-version-check', '--progress-bar=off', 'requestss']' returned non-zero exit status 1.

In [28]:
import subprocess
'FCA_base' in str(subprocess.run('bash -c "conda env list"', capture_output=True, shell=True))

True

In [125]:
# fa2 is required, because sctk.fdg (_fdg.py) does not currently does not 
# currently allow "fr" layout, returned by scanpy.tl.draw_graph.
init_package_ls = {'numpy', 'scanpy', 'anndata', 'sctk', 'cellrank', 'scvelo', 
              'seaborn', 'matplotlib', 'pandas', 'numpy_groupies', 
              'scikit-learn', 'plotnine', 'fa2', 'scirpy', 'harmonypy', 'umap-learn'}
package_versions = dict()
channels = []
env, channels = load_env(str(Path(root, 'logs', 'FCA_init.yml').resolve()))
for p in init_package_ls:
    if not any(p==pkg for pkg in env['package']):
        print(f'{p} not found in the .yaml file')
    else:
        package_versions[p] = env['version'][env['package'].index(p)]

sctk not found in the .yaml file
scirpy not found in the .yaml file


In [60]:
channels

['conda-forge', 'bioconda', 'defaults']

- plotnine can be installed with ```conda install -c conda-forge plotnine```

- SCTK can currently only be installed by```pip install``` exclusively.

- SCTK was cloned from git by ```git clone git@github.com:Teichlab/sctk.git```.

- plotnine was added to ```package_versions``` without a version (channel: conda-forge)

In [61]:
package_versions

{'harmonypy': '0.0.5',
 'seaborn': '0.10.1',
 'numpy_groupies': '0.9.13',
 'matplotlib': '3.3.2',
 'umap-learn': '0.4.6',
 'anndata': '0.7.5',
 'scvelo': '0.2.2',
 'scanpy': '1.7.0',
 'numpy': '1.19.1',
 'fa2': '0.3.5',
 'pandas': '1.1.2',
 'scikit-learn': '1.0.2',
 'cellrank': '1.0.0',
 'plotnine': '0.7.1'}

<span style='color:salmon'> The environment with these specified dependencies was causing issues with reading of  ```.h5ad``` files; converting the strings, namely the column names, into bytes, i.e. ```b"string"``` format. <br /> This is a know issue caused by ```h5py>2.10.0``` mixed with ```anndata=0.7.x```. To fix, ```h5py=2.10.0``` is manually added to the ```package_versions``` (and ```fussy_deps```) and to [follow tested solutions](https://github.com/scverse/anndata/issues/442) ```~/FCA_Fetal_Skin_priv/logs/FCA_init.yml``` was modified by changing ```Anndata=0.7.4``` to ```Anndata=0.7.5``` <b>before running the following</b>. </span>

<span style='color:salmon'> Furthermore, as ```SCTK``` (and subsequently its dependencies) must be installed exclusively through ```pip```, measures are taken to avoid the conflicts often caused by using ```pip``` in and environment managed by ```conda```.</span>:

- [X] Dependencies which are already installed in the conda env are isolated (```adjustText=0.7.3, igraph=0.10.2, leidenalg=0.9.0, texttable-1.6.4```), manually added (see below) to ```package_versions``` (and ```fussy_deps```) and consequently installed via conda. 
- [X] ```SCTK``` is ```git clone```ed and installed via ```.../sctk$ pip install --no-deps .```.

In [126]:
package_versions['h5py']='==2.10.0'
package_versions['adjustText']='==0.7.3'
package_versions['igraph']='==0.10.2'
package_versions['leidenalg']='==0.9.0'
package_versions['texttable']='==1.6.4'
package_versions['scirpy']=''
package_versions['umap-learn']='==0.5.1' # Most up-to-date version, which doesn't break the umap manifold. Ignoring, the one found in FCA_init.yml
package_versions

{'harmonypy': '==0.0.5',
 'seaborn': '==0.10.1',
 'numpy_groupies': '==0.9.13',
 'matplotlib': '==3.3.2',
 'umap-learn': '==0.5.1',
 'anndata': '==0.7.5',
 'scvelo': '==0.2.2',
 'scanpy': '==1.7.0',
 'numpy': '==1.19.1',
 'fa2': '==0.3.5',
 'pandas': '==1.1.2',
 'scikit-learn': '==1.0.2',
 'cellrank': '==1.0.0',
 'plotnine': '==0.7.1',
 'h5py': '==2.10.0',
 'adjustText': '==0.7.3',
 'igraph': '==0.10.2',
 'leidenalg': '==0.9.0',
 'texttable': '==1.6.4',
 'scirpy': ''}

In [131]:
subprocess.run(["conda", "", "/dev/null"], capture_output=True)

FileNotFoundError: [Errno 2] No such file or directory: 'lsd'

In [127]:
build_sub_env(str(Path(root, 'logs', 'FCA_init.yml').resolve()), 
              package_versions, 
              'FCA_base_t', constraint='SoE')

mamba create -n FCA_base_t -c conda-forge -c bioconda -c defaults harmonypy==0.0.5 seaborn==0.10.1 numpy_groupies==0.9.13 matplotlib==3.3.2 umap-learn==0.5.1 anndata==0.7.5 scvelo==0.2.2 scanpy==1.7.0 numpy==1.19.1 fa2==0.3.5 pandas==1.1.2 scikit-learn==1.0.2 cellrank==1.0.0 plotnine==0.7.1 h5py==2.10.0 adjustText==0.7.3 igraph==0.10.2 leidenalg==0.9.0 texttable==1.6.4 scirpy


In [155]:
fussy_deps = ['anndata', 'scanpy', 'h5py',
              'adjustText','igraph','leidenalg',
              'texttable', 'umap-learn', 'harmonypy'] # list of version-specific packages

In [10]:
with Path(root, 'logs', 'FCA_base.sh').open(mode='w') as conda_install:
    conda_install.write(f'mamba create -n FCA_base')
    conda_install.write(" ")
    for ch in channels:
        conda_install.write(f'-c {ch}')
        conda_install.write(" ")
    for pack in package_versions:
        if pack in fussy_deps:
            conda_install.write(f'{pack}={package_versions[pack]}')
            conda_install.write(" ")
        else:
            conda_install.write(f'{pack}')
            conda_install.write(" ")

In [11]:
with Path(root, 'logs', 'FCA_base.sh').open(mode='r') as conda_install:
    for line in conda_install:
        print(line)

mamba create -n FCA_base -c conda-forge -c bioconda -c defaults -c conda-forge -c bioconda -c defaults anndata=0.7.5 cellrank fa2 harmonypy=0.0.5 matplotlib numpy numpy_groupies pandas plotnine scanpy=1.7.0 scikit-learn scvelo seaborn umap-learn=0.5.1 h5py=2.10.0 adjustText=0.7.3 igraph=0.10.2 leidenalg=0.9.0 texttable=1.6.4 scirpy 


In [12]:
# !chmod +x ~/FCA_Fetal_Skin_priv/logs/FCA_base.sh

In [13]:
# !~/FCA_Fetal_Skin_priv/logs/FCA_base.sh

In [14]:
# !conda activate FCA_base
# !cd ~/sctk
# !pip install . --no-deps
# !conda env export > ~/FCA_Fetal_Skin_priv/logs/FCA_base.yml

To cross-ref two ```conda envs```

In [67]:
p = Path(r'/home/jovyan/my-conda-envs/monocle3/conda-meta').expanduser().glob('r-monocle3*.json')
files = [str(x) for x in p if x.is_file()]
files

['/home/jovyan/my-conda-envs/monocle3/conda-meta/r-monocle3-1.0.0-r41h7d875b9_1.json']

In [48]:
test_str = 'r-monocle3-1.0.0-r41h7d875b9_1.json'
re.search(f'r-monocle.*.json', test_str).group(0)

'r-monocle3-1.0.0-r41h7d875b9_1.json'

In [80]:
def installed_package_reqs(env_dir, package_name, full_json_path=None):
    """Takes the path to the target conda env and query package and 
    extract the packages the query package depends on in that env."""
    
    import json
    from collections import defaultdict

    req_dir = Path(env_dir, 'conda-meta').expanduser().glob(f'{package_name}*.json')
    req_file = [str(x) for x in req_dir if x.is_file()][0]
    req_file = json.load(open(req_file))
    requirements = defaultdict(set)
    for k, v in req_file.items():
        if 'depends' in k:
            for r in req_file[k]:
                try:
                    requirements[r.split(' ')[0]].add(r.split(' ')[1])
                except IndexError:
                    requirements[r.split(' ')[0]].add('')
    
    return requirements

In [100]:
installed_package_reqs('/home/jovyan/my-conda-envs/FCA_base/', 'pandas')

defaultdict(set,
            {'libgcc-ng': {'>=12'},
             'python': {'>=3.9,<3.10.0a0'},
             'python_abi': {'3.9.*'},
             'libstdcxx-ng': {'>=12'},
             'numpy': {'>=1.20.3,<2.0a0'},
             'python-dateutil': {'>=2.8.1'},
             'pytz': {'>=2020.1'}})

In [40]:
crossref_2_envs(str(Path(root, 'logs', 'FCA_base_monocle3_sb.yml').resolve()), 
                str(Path(root, 'logs', 'monocle3.yml').resolve()), 
                dependencies+['r-monocle3'], compare_build=True) # try compare_build=True if you think the build difference maybe important



                                               FCA_base_monocle3_sb                                       monocle3

                                          version           build                              version            build

_libgcc_mutex                              0.1             conda_forge                          0.1             conda_forge
_openmp_mutex                              4.5             2_gnu                                4.5             1_gnu
_r-mutex                                   1.0.1           anacondar_1                          1.0.1           anacondar_1
adjusttext                                 0.7.3.1         py_1                                 null            null
airr                                       1.4.1           pyh5e36f6f_0                         null            null
alsa-lib                                   1.2.8           h166bdaf_0                           null            null
anndata                                    0

```sceasy``` not found in ```monocle3```!

Do the above process done for ```FCA_base``` for ```monocle3``` but this time add all the packages to the ```fussy_deps``` and then add ```r-sceasy``` without a specified version to the created ```monocle3.sh``` file. 

In [234]:
import json

import requests

package_name = 'tensorflow'
url = 'https://pypi.python.org/pypi/' + str(package_name) + '/json'
data = requests.get(url).json()

print(data['info']['requires_dist'])

['absl-py (>=1.0.0)', 'astunparse (>=1.6.0)', 'flatbuffers (>=2.0)', 'gast (<=0.4.0,>=0.2.1)', 'google-pasta (>=0.1.1)', 'h5py (>=2.9.0)', 'libclang (>=13.0.0)', 'numpy (>=1.20)', 'opt-einsum (>=2.3.2)', 'packaging', 'protobuf (<3.20,>=3.9.2)', 'setuptools', 'six (>=1.12.0)', 'termcolor (>=1.1.0)', 'typing-extensions (>=3.6.6)', 'wrapt (>=1.11.0)', 'grpcio (<2.0,>=1.24.3)', 'tensorboard (<2.12,>=2.11)', 'tensorflow-estimator (<2.12,>=2.11.0)', 'keras (<2.12,>=2.11.0)', 'tensorflow-io-gcs-filesystem (>=0.23.1) ; platform_machine != "arm64" or platform_system != "Darwin"']
